# Gramener Case Study

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 20

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### External contants

In [ ]:
loan_file_path="./loan/loan.csv"
data_dict_file_path = "./loan/Data_Dictionary.xlsx"

## Check Point 1:  Data Cleaning

### - Fetching ```loan``` data from given file

In [ ]:
loan = pd.read_csv(loan_file_path, encoding="utf8")
loan.head()

#### Finding number of rows and columns in the dataset

In [ ]:
print('The loan dataset set has %d rows and %d columns' % (loan.shape[0], loan.shape[1]))

#### Removing columns which have all values as ```NaN```

In [ ]:
nan_count_df = loan.isnull().sum().to_frame('nan_count')
num_col_with_all_nan = nan_count_df.loc[nan_count_df['nan_count']  == loan.shape[0], :].shape[0]
print('Number of columns with all NaN values are %d.' % (num_col_with_all_nan))

In [ ]:
# Removing columns which have all values as NaN
loans_frame = loan.dropna(axis=1, how="all")
print('The loan dataset set has now %d rows and %d columns' % (loans_frame.shape[0], loans_frame.shape[1]))

### - Fetching ```Data Dictionary```  from given file

In [ ]:
data_dict = pd.read_excel(data_dict_file_path, sheet_name="LoanStats")
data_dict.head()

#### Initial List of columns for consideration 

In [ ]:
filtered_data_dict = data_dict.loc[data_dict['LoanStatNew'].isin(loans_frame.columns), :]

### Considering those loan entries which are  either `Fully Paid` or `Charged Off`  to discover any pattern in them
The reason for not selecting `Current` loans because we don't if they will be fully paid or default in future. After creating a model we can predict if `Current` loans will default or not.

In [ ]:
loans_frame = loans_frame.loc[loans_frame['loan_status'].isin(['Fully Paid', 'Charged Off']), :]
loans_frame.head()

In [ ]:
# Shape of the 'Charged Off' records i.e. number of records and columns
loans_frame.shape

In [ ]:
clean_loan_df = loans_frame.isnull().sum().sort_index().to_frame('nan_count').reset_index()

In [ ]:
# Creating a dataframe by merging the data dictional the clean_loan dataframe to get a better understanding
column_desc = pd.merge(filtered_data_dict, clean_loan_df, how="inner", left_on="LoanStatNew", right_on="index")
column_desc.loc[:,['LoanStatNew','Description','nan_count']]

In [ ]:
column_desc.loc[column_desc['nan_count'] > 0,['LoanStatNew','Description','nan_count']]

##### Removing columns  
- ```mths_since_last_delinq```, ```mths_since_last_record```, ```next_pymnt_d``` as the contain more 50% values as NaN
- ```emp_title```, ```desc```, ```title```, ```url``` are irrelevant for EDA

In [ ]:
#### Removing columns  mths_since_last_delinq, mths_since_last_record, next_pymnt_d, emp_title, desc
columns_to_remove = ['emp_title', 'desc', 'url', 'title']
columns_under_process = column_desc.loc[~column_desc['LoanStatNew'].isin(columns_to_remove), 'LoanStatNew'].values
loans_frame = loans_frame.loc[:,columns_under_process]

# Validate will take 50% percent as cutoff and will keep below that
max_allowed_nan = len(loans_frame)/2
print(max_allowed_nan)
loans_frame = loans_frame.loc[:,(nan_count_df['nan_count'] < max_allowed_nan)]
print(loans_frame.isnull().sum())

In [ ]:
loans_frame.shape

In [ ]:
print("Total number of unique member in the data set is %d" % loans_frame['member_id'].nunique())

In [ ]:
pd.options.display.max_columns = 100 # To see all columns
loans_frame.describe()

In [ ]:
# Validate all values in column is same by checking unique values in clumn
uniques = loans_frame.apply(lambda x:x.nunique())
print(uniques)

In [ ]:
# Remove columns where unique values are 1 as it illustrates that all values are same in column
loans_frame = loans_frame.drop(uniques[uniques <= 1].index, axis = 1)
print(loans_frame.shape)
loans_frame.head()

In [ ]:
column_desc.loc[~column_desc['LoanStatNew'].isin(columns_to_remove), ['LoanStatNew', 'Description']]

### Check Point 2: Univariate Analysis

From the above results we can now chalk out catergorical columns in the ```loans_frame```
- ```unordered categorical variables``` 
==> addr_state, application_type, home_ownership, initial_list_status, purpose, pymnt_plan, sub_grade, verification_status, zip_code
- ```ordered categorical variables```
==> emp_length, grade, sub_grade, term

In [ ]:
def category_univariate_analysis(column_name):
    df = loans_frame.groupby('loan_status')[column_name].value_counts(ascending=False)\
                            .unstack(level=0).reset_index()
    df['Charged Off'].fillna(value=0, inplace=True)
    df['Total'] = df['Charged Off'] + df['Fully Paid']
    df['Charged_Off_Percent'] = df['Charged Off'] * 100 /df['Total']
    df['Fully_Paid_Percent'] = df['Fully Paid'] * 100 /df['Total']
    return df.sort_values(by='Charged_Off_Percent', ascending=False)

In [ ]:
def power_law_df(df, column_name):
    s1 = np.log(df['Charged Off'].rank(axis=0, ascending=False)).to_frame(name='Rank_Log')
    s2 = np.log(df['Charged Off']).to_frame(name='Count_Log')
    s3 = df[column_name].to_frame(name=column_name)

    power_law_1 = pd.concat([s1, s2, s3], axis=1)
    power_law_1['status'] = 'Charged Off'

    s1 = np.log(df['Fully Paid'].rank(axis=0, ascending=False)).to_frame(name='Rank_Log')
    s2 = np.log(df['Fully Paid']).to_frame(name='Count_Log')

    power_law_2 = pd.concat([s1, s2, s3], axis=1)
    power_law_2['status'] = 'Fully Paid'

    power_law = pd.concat([power_law_1, power_law_2], ignore_index=True)
    return power_law

In [ ]:
loans_frame['verification_status'].value_counts(normalize=True)

In [ ]:
verification_status_analysis = category_univariate_analysis('verification_status')
verification_status_analysis

In [ ]:
df = power_law_df(verification_status_analysis, 'verification_status')
df.sort_values(by='Rank_Log')

In [ ]:
sns.pointplot(x="Rank_Log", y="Count_Log", hue="status", data=df.loc[:,['Rank_Log', 'Count_Log', 'status']])
plt.show()

In [ ]:
sns.barplot(x="verification_status", y="Charged_Off_Percent", data=verification_status_analysis)\
    .set(ylabel='Charged Off(%)', xlabel='Verification Status')
plt.show()

In [ ]:
loans_frame['home_ownership'].value_counts(normalize=True)

In [ ]:
home_ownership_analysis = category_univariate_analysis('home_ownership')
home_ownership_analysis

In [ ]:
df = power_law_df(home_ownership_analysis, 'home_ownership')
df.sort_values(by='Rank_Log')

In [ ]:
sns.pointplot(x="Rank_Log", y="Count_Log", hue="status", data=df.loc[:,['Rank_Log', 'Count_Log', 'status']])
plt.show()

In [ ]:
sns.barplot(x="home_ownership", y="Charged_Off_Percent", data=home_ownership_analysis)\
    .set(ylabel='Charged Off(%)', xlabel='Home Ownership')
plt.show()

In [ ]:
loans_frame['purpose'].value_counts(normalize=True)

In [ ]:
purpose_analysis = category_univariate_analysis('purpose')
purpose_analysis

In [ ]:
df = power_law_df(purpose_analysis, 'purpose')
df.sort_values(by='Rank_Log')

In [ ]:
sns.pointplot(x="Rank_Log", y="Count_Log", hue="status", data=df.loc[:,['Rank_Log', 'Count_Log', 'status']])
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
g = sns.barplot(x="purpose", y="Charged_Off_Percent", data=purpose_analysis)\
    .set(ylabel='Charged Off(%)', xlabel='Purpose')
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
plt.show()

In [ ]:
loans_frame['grade'].value_counts(normalize=True)

In [ ]:
grade_analysis = category_univariate_analysis('grade')
grade_analysis

In [ ]:
plt.figure(figsize=(15,12))
plt.subplot(3, 1, 1)
g = sns.barplot(x="grade", y="Charged_Off_Percent", data=grade_analysis.sort_values(by='grade'))\
    .set(ylabel='Charged Off(%)', xlabel='Grade')
    
plt.subplot(3, 1, 2)
sns.barplot(x="grade", y="Charged Off", data=grade_analysis)\
    .set(ylabel='Charged Off(Count)', xlabel='Grade')

plt.subplot(3, 1, 3)
sns.barplot(x="grade", y="Fully Paid", data=grade_analysis)\
    .set(ylabel='Fully Paid(Count)', xlabel='Grade')
plt.show()

In [ ]:
category_univariate_analysis('sub_grade')

In [ ]:
loans_frame['emp_length'].value_counts(normalize=True)

In [ ]:
emp_length_analysis = category_univariate_analysis('emp_length')
emp_length_analysis

In [ ]:
plt.figure(figsize=(15,12))
plt.subplot(3, 1, 1)
sns.barplot(x="emp_length", y="Charged_Off_Percent", data=emp_length_analysis.sort_values(by='emp_length'))\
    .set(ylabel='Charged Off(%)', xlabel='Employee work experience')
    
plt.subplot(3, 1, 2)
sns.barplot(x="emp_length", y="Charged Off", data=emp_length_analysis.sort_values(by='emp_length'))\
    .set(ylabel='Charged Off(Count)', xlabel='Employee work experience')

plt.subplot(3, 1, 3)
sns.barplot(x="emp_length", y="Fully Paid", data=emp_length_analysis.sort_values(by='emp_length'))\
    .set(ylabel='Fully Paid(Count)', xlabel='Employee work experience')
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
plt.show()

In [ ]:
loans_frame['term'].value_counts(normalize=True)

In [ ]:
term_analysis = category_univariate_analysis('term')
term_analysis

In [ ]:
g = sns.barplot(x="term", y="Charged_Off_Percent", data=term_analysis)\
    .set(ylabel='Charged Off(%)', xlabel='Term')
plt.show()

In [ ]:
loans_frame['addr_state'].value_counts(normalize=True)

In [ ]:
addr_state_analysis = category_univariate_analysis('addr_state')
addr_state_analysis

In [ ]:
plt.figure(figsize=(15,4))
g = sns.barplot(x="addr_state", y="Charged_Off_Percent", data=addr_state_analysis)\
    .set(ylabel='Charged Off(%)', xlabel='Address State')
# locs, labels = plt.xticks()
# plt.setp(labels, rotation=45)
plt.show()

#### Quatitative variable univariate analysis

In [ ]:
loans_frame['revol_util'] = loans_frame['revol_util'].astype('str')
loans_frame['revol_util'] = loans_frame['revol_util'].map(lambda x : x.rstrip('%'))
loans_frame['revol_util'] = loans_frame['revol_util'].astype('float')
bins=[0,10,20,30,40,50,60,70,80,90,100]
pd.cut(loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','revol_util'], bins=bins).value_counts(normalize=True)

In [ ]:
loans_frame['revol_util'] = loans_frame['revol_util'].astype('str')
loans_frame['revol_util'] = loans_frame['revol_util'].map(lambda x : x.rstrip('%'))
loans_frame['revol_util'] = loans_frame['revol_util'].astype('float')
bins=[0,10,20,30,40,50,60,70,80,90,100]
pd.cut(loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','revol_util'], bins=bins).value_counts(normalize=True)

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','revol_bal'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','revol_bal'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','dti'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','dti'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','open_acc'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','open_acc'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','total_acc'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','total_acc'].describe()

In [ ]:
def dateformat(earliest_cr_line_date):
    date_split = earliest_cr_line_date.split('-')
    if int(date_split[1]) > 18:
        date_split[1] = '19' +  date_split[1]
    else:
        date_split[1] = '20' +  date_split[1]
    return '-'.join(date_split)
loans_frame['earliest_cr_line_mod'] = loans_frame['earliest_cr_line'].apply(dateformat)

In [ ]:
loans_frame['credit_history'] = round((pd.to_datetime(loans_frame['issue_d'], format='%b-%y') - \
                                pd.to_datetime(loans_frame['earliest_cr_line_mod'], format='%b-%Y')) / np.timedelta64(1, 'M'))
loans_frame['credit_history'].head()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','credit_history'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','credit_history'].describe()

In [ ]:
loans_frame.loc[:,['annual_inc','installment','revol_bal']]
loans_frame['new_debt_annual_inc_ratio'] = (loans_frame['installment'] * 12 + loans_frame['revol_bal']) /\
                                            loans_frame['annual_inc']
loans_frame['new_debt_annual_inc_ratio'].head()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Charged Off','new_debt_annual_inc_ratio'].describe()

In [ ]:
loans_frame.loc[loans_frame['loan_status'] == 'Fully Paid','new_debt_annual_inc_ratio'].describe()

In [ ]:
loans_frame.shape

## Bivariate Analysis
##### Let see co relation between columns with charged off and fully paid customers. 

In [ ]:
loans_frame.head()

In [ ]:
# Remove % from int_rate for performing co relation
loans_frame['int_rate'] = loans_frame['int_rate'].apply(lambda x: float(x.split('%')[0]))
loans_frame.head()

In [ ]:
# Function to drive a column named charged_off with 1 stating as customer charged off and 0 stating as customer not charged off
def addChargedOff(row):
    if(row['loan_status'] == 'Charged Off'):
        val = 1
    else :
        val = 0
    return val

# Function to drive a column named fully_paid with 1 stating as customer fully paid loan and 0 stating as customer not fully paid
def addFullyPaid(row):
    if(row['loan_status'] == 'Fully Paid'):
        val = 1
    else :
        val = 0
    return val
        
    

In [ ]:
# Adding columns with the name of charged_off and fully_paid
loans_frame_without_current = loans_frame.loc[loans_frame['loan_status'] != 'Current']
loans_frame_without_current['charged_off'] = loans_frame_without_current.apply(addChargedOff,axis = 1)
loans_frame_without_current['fully_paid'] = loans_frame_without_current.apply(addFullyPaid, axis = 1)
loans_frame_without_current.head()

In [ ]:
# Driving corelation of charged_off and fully_paid with other columns.
corr = loans_frame_without_current.corr()
corr

In [ ]:
# Drawing heatmap to see pattern of corelation effecting loan getting default or fully paid
loans_frame_without_current = loans_frame_without_current[['charged_off','fully_paid','annual_inc','dti','inq_last_6mths','installment','int_rate','loan_amnt','pub_rec_bankruptcies','revol_bal','revol_util','total_acc','total_rec_int','total_rec_late_fee','total_rec_prncp','credit_history','new_debt_annual_inc_ratio']].copy()
plt.figure(figsize = (20,20))
sns.heatmap(loans_frame_without_current.corr(),annot=True,cmap='viridis_r')
plt.show()

## Conclusion:
Above trend shows co relation inwhich negative means decrease in attribute value increase the probability of the event and positve means increase in attribute value increase the probability of the event.

###### Below are reasons for probability of loan getting default / charged off
###### Low Annual Income
###### High DTI
###### High installments
###### High interest rate
###### High Loan amount 
###### High frequency of recieving of late fees
###### Less availbility of credit history


These above variables most probably influences loan success / unsuccess.
